In [23]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc.conversions import mass1_from_mchirp_eta, mass2_from_mchirp_eta, tau0_from_mass1_mass2, tau3_from_mass1_mass2, mass1_from_mchirp_q, mass2_from_mchirp_q, mass1_from_tau0_tau3, mass2_from_tau0_tau3, q_from_mass1_mass2, eta_from_q
import random
from pycbc.filter import sigmasq
from scipy.linalg import svd
from scipy.interpolate import CubicSpline, CubicHermiteSpline, interp1d, BarycentricInterpolator, krogh_interpolate, PchipInterpolator, Akima1DInterpolator
from scipy import special
from rbf.interpolate import RBFInterpolant
#from rbf.interpolate import _objective
from rbf.poly import mvmonos
from pycbc.detector import Detector
import h5py
import pickle
import matplotlib
from pycbc.types.timeseries import TimeSeries
from pycbc.types.array import complex_same_precision_as
from pycbc.conversions import mchirp_from_mass1_mass2
from pycbc.pnutils import get_final_freq
from pycbc.waveform import get_fd_waveform
from pycbc.filter.matchedfilter import get_cutoff_indices
from pycbc.fft import ifft
from pycbc.types import zeros
from pycbc.types.frequencyseries import load_frequencyseries
from pycbc.types.timeseries import load_timeseries
from tqdm import tqdm
from multiprocessing import Process
from multiprocessing.pool import Pool
from pycbc.inference.models.marginalized_gaussian_noise import MarginalizedPhaseGaussianNoise

from gwosc.datasets import event_gps
from pycbc.waveform.generator import FDomainDetFrameGenerator, FDomainCBCGenerator
import statistics
from functools import partial
from concurrent.futures import ProcessPoolExecutor
from pycbc import distributions
from scipy.stats import truncnorm
import copy
from gwfast.waveforms import TaylorF2_RestrictedPN, IMRPhenomD_NRTidalv2, IMRPhenomD
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams
import gwfast.gwfastGlobals as glob
from scipy.stats import gaussian_kde
from fisherTools import dMc_dm1, dMc_dm2


from pycbc.frame.frame import read_frame
from pycbc.filter import matched_filter, highpass, lowpass
import matplotlib.pyplot as plt
from pycbc.psd import interpolate, welch, inverse_spectrum_truncation
from gwosc.datasets import event_gps
import numpy as np
from pycbc.waveform.generator import FDomainDetFrameGenerator, FDomainCBCGenerator
from pycbc.waveform.waveform import get_fd_waveform, get_waveform_filter_length_in_time
from pycbc.catalog import Merger
from scipy import signal 
from pycbc.types.timeseries import TimeSeries
from pycbc.pnutils import get_final_freq

from pycbc import frame
from pycbc import noise
import pycbc.psd
from pycbc.pnutils import f_SchwarzISCO

In [24]:
f_low = 20  # low cutoff frequency

f_high = 1600 # Largest frequency present in chirp signal or high cutoff frequency

filter_order = 8


tc = event_gps('GW170817') # Merger Time

ifo = ['L1', 'H1', 'V1']
print(tc)

1187008882.4


In [25]:

data = {}
for i in range(len(ifo)):
    
    data[ifo[i]] = read_frame("{}-{}_LOSC_CLN_4_V1-1187007040-2048.gwf".format(ifo[i][0], ifo[i]),\
                    '{}:LOSC-STRAIN'.format(ifo[i]),
                   start_time= tc + 1,
                   end_time= tc + 100,
                   check_integrity=False)
    

In [26]:
psd = {}

for i in range(len(ifo)):
    
    psd[ifo[i]] = data[ifo[i]].psd(2, avg_method = 'median-mean')
    psd[ifo[i]] = interpolate(psd[ifo[i]], data[ifo[i]].to_frequencyseries().delta_f)
    psd[ifo[i]] = inverse_spectrum_truncation(psd[ifo[i]],\
                                               int(2 * psd[ifo[i]].sample_rate), low_frequency_cutoff=15, trunc_method='hann')
    #psd[ifo[i]].save(f'PSD_{ifo[i]}.txt')

In [27]:
delta_t = data[ifo[0]].delta_t
print(delta_t, 1/delta_t)
tsamps = int(data[ifo[0]].duration/delta_t)
noise_det = {}
for i in range(len(ifo)):
    noise_det[ifo[i]] = noise.gaussian.frequency_noise_from_psd(psd[ifo[i]], seed = 4)
    


0.000244140625 4096.0


In [ ]:
print(noise_det[ifo[0]])

In [35]:
np.random.seed(0)
params_sim = []
while True:
    m1 = (2.35 - 1)*np.random.rand(1) + 1
    m2 = (2.35 - 1)*np.random.rand(1) + 1
    chi1z = (0.05 - (-0.05))*np.random.rand(1) + (-0.05)
    chi2z = (0.05 - (-0.05))*np.random.rand(1) + (-0.05)
    distance = (200 - 10)*np.random.rand(1) + 10
    inclination = (2*np.pi  - 0)*np.random.rand(1) + 0
    ra = (2*np.pi - 0)*np.random.rand(1) + 0
    dec = (np.pi/2 - (-np.pi/2))*np.random.rand(1) + (-np.pi/2)
    
    
    params = {'approximant' : 'TaylorF2', 'mass1': m1[0], 'mass2': m2[0], 'spin1z': chi1z[0],\
          'spin2z': chi2z[0], 'distance': distance[0], 'ra': ra[0], 'dec': dec[0], \
          'inclination': inclination[0], 'tc': 1187008932.9, 'polarization': 0}
    generator = FDomainDetFrameGenerator(FDomainCBCGenerator,\
                                         epoch = data[ifo[0]].start_time,\
                                         detectors = ifo, **params, f_lower = f_low,\
                                         delta_f = 1/data[ifo[0]].duration)
    signal_det = generator.generate()
    data_sim = {}
    for i in range(len(ifo)):
        signal_det[ifo[i]].resize(len(noise_det[ifo[i]]))
        data_sim[ifo[i]] = signal_det[ifo[i]] + noise_det[ifo[i]]
        
    hp, hc = get_fd_waveform(approximant = params['approximant'], mass1 = params['mass1'], mass2 = params['mass2'], spin1z = params['spin1z'], spin2z = params['spin2z'],f_lower = f_low,\
                        delta_f = data_sim[ifo[0]].delta_f)
    hp.resize(len(data_sim[ifo[0]]))
    net_snr = 0
    for i in range(len(ifo)):
        snr = matched_filter(hp, data_sim[ifo[i]], psd=psd[ifo[i]], low_frequency_cutoff= 20.0)
        snr = snr.crop(25, 0)
        net_snr += abs(snr).max()**2
    net_snr = np.sqrt(net_snr)
    
    if 10 < net_snr < 100:
        params_sim.append([m1[0], m2[0], chi1z[0], chi2z[0], distance[0], inclination[0], ra[0], dec[0],net_snr])
    
    if len(params_sim) == 100:
        break

In [36]:
print(np.array(params_sim)[:, 8])

[12.0743606  10.54098615 16.43387278 10.82679781 45.5783955  21.87330087
 15.2713294  22.4035667  14.73358542 17.98318697 10.02007671 42.46367275
 44.75669395 13.18353435 19.34380926 12.27634724 31.5461584  12.64769164
 13.43744749 11.65959764 18.12686922 23.41200971 11.45111655 15.99193083
 18.62121782 12.59495221 11.03368518 11.60601983 19.49287647 10.36322515
 12.2642847  19.36143482 22.61158334 19.30750908 12.52076911 29.05110608
 13.77106115 21.1838646  20.58164301 79.56992069 12.72692453 19.09029711
 13.27755781 29.3960433  28.59278034 57.76237753 24.03875889 10.50495166
 11.03179714 15.91377781 14.8013006  15.33313716 10.9852242  39.05666616
 13.88616463 51.6010091  21.28626432 20.70726668 45.06154288 11.405732
 19.98640412 10.99943176 19.32278078 38.95547055 17.16896618 12.16171799
 15.73525692 13.87878681 17.42745321 63.1825368  11.01062497 12.53336126
 19.59563917 37.38142603 19.42476596 10.84623549 11.94130605 14.16787439
 19.94569489 66.30981145 21.39832484 18.72354596 10.1